In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import re
import requests
import urllib
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import nltk
from nltk.corpus import stopwords
import pymorphy2
import pyLDAvis
import pyLDAvis.gensim
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
#nltk.download('stopwords')
#nltk.download('punkt')

C:\Programms\Anaconda\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Programms\Anaconda\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/dev

In [2]:
stop_words = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

"""Очистка строки
input: строка с "грязными" данными
output: "чистая" строка
"""

def clean_string(string):
    dirt_patterns = [r'\'', r'\\n', r'\n', r'Б\\\\/Н', r'\\\\/\d+',r'\\\\/', r'\\{1,}',r'\(.*\)',r'c\.$',r'ул\.$',r'-\d+',r'пер\.$', r'д\.']
    for pattern in dirt_patterns:
        string = re.sub(pattern,'',string)
    return string


"""Поиск темы
input: строка для поиска темы
output: тема
"""

def search_theme(string):
    res = re.findall(r'«.*»',string)
    return res[0] if len(res)>0 else ""


"""Поиск адреса
input: строка для поиска адреса
output: адрес объекта
"""
def search_address(string):
    res = re.findall(r'по адресу\s?(.*$)',string)
    return res[0] if len(res)>0 else ""


"""Производит предварительную обработку текста
input: текст для обработки
output: ообработанный текс (без пунктуации, слова приведены к нормальной форме, числовые данные удалены)
"""

def preprocessingText(line):
    line = line.lower()
    line = re.sub(r'\d+','',line,flags=re.UNICODE)#delete numbers
    line = re.sub(r'[^\w\s]',' ',line,flags=re.UNICODE)
    tl = nltk.word_tokenize(line) #tl - tokenize line
    nft = [morph.parse(token)[0].normal_form for token in tl if len(token)>1]#nft - normal form token
    clean_words_list = [normalToken for normalToken in nft if normalToken not in stop_words and len(normalToken)>1]#преобразованный список
    return ' '.join(clean_words_list)


"""Приведение даты в необходимый формат
input:строка, содержащая дату
output: строка даты в формате dd.MM.yyyy hh:mm:ss
"""

def prepare_date(date_string):
    months = {'Января':'01','Февраля':'02','Марта':'03','Апреля':'04',
                'Мая':'05','Июня':'06','Июля':'07','Августа':'08',
                'Сентября':'09','Октября':'10','Ноября':'11','Декабря':'12'}
    date_string = re.sub(r'\s{2,}',' ',str(date_string.strip()))
    try:   
        split_date_string = date_string.split(' ')
        day = split_date_string[0]
        month = months[split_date_string[1]]
        year = split_date_string[2]
        time = split_date_string[4]
        full_date_string = "{0}.{1}.{2} {3}:00".format(day,month,year,time)      
    except Exception as error:
        full_date_string = date_string
    return full_date_string


"""Поиск округа
input: строка, в которой может содержаться округ
output: округ или строка, подававшаяся на вход, в зависимости от результата поиска
"""

def search_district(full_address):
    searched = re.findall(r'округ\s+?(.+?),',full_address)
    result = full_address if len(searched)==0 else searched[0]
    return result

In [ ]:
separator = r'\', \''
frame_columns_names = ["Тема обращения","Суть обращения",
                        "Чиновник","Должность",
                        "Дата подачи заявления"]
df = pd.read_csv('Mosru_dump_all_pages.csv',
                    encoding = 'utf-8',
                    sep = separator,
                    names = None,
                    skiprows = 1)
df.columns = frame_columns_names

In [ ]:
for column in frame_columns_names:
    df[column] = df[column].apply(clean_string)

In [ ]:
df["Адрес"] = df['Тема обращения'].apply(search_address)

In [ ]:
df["Тема обращения"] = df['Тема обращения'].apply(search_theme)

In [ ]:
df['Обращение нормализованное'] = df['Суть обращения'].apply(preprocessingText)

In [ ]:
df["Дата подачи заявления"] = df['Дата подачи заявления'].apply(prepare_date)

In [ ]:
df.to_csv('clean_frame.csv',sep='|',encoding='utf-8')

Вторая часть после загрузки и обработки данных (подтянуть адреса c помощью mapbox api)

In [ ]:
n_df = pd.read_csv('clean_frame.csv', sep='|', encoding='utf-8')

In [ ]:
unique_addresses = n_df['Адрес'].unique().tolist()

In [ ]:
mapbox_token = "pk.eyJ1IjoidmFkaW05NiIsImEiOiJja25nMGowN2QyNnFlMnFtdTkzYjBjejdkIn0.18dpTc8Vgja2gPDUYgrSkw"

In [ ]:
def search_district(address):
    url = "https://api.mapbox.com/geocoding/v5/mapbox.places/{0}.json?access_token={1}".format(urllib.parse.quote("Москва "+str(address)), mapbox_token)
    response = requests.get(url)
    district = ""
    try:
        district = response.json()['features'][0]['place_name']
    except Exception as error:
        print("Возникла ошибка-{0}, адрес - {1}".format(str(error),str(address)) )
    finally:
        return district

In [ ]:
with open('unique_addresses_frame.csv','w',encoding='utf-8') as file:
    file.write('Адрес|Полный ответ\n')
    for address in tqdm(unique_addresses):
        full_response = search_district(address)
        file.write('{0}|{1}\n'.format(address,full_response))

Схлестываем значения адресов с "чистым фреймом"

In [10]:
addresses_frame = pd.read_csv('unique_addresses_frame.csv',encoding='utf-8',sep='|')

In [11]:
clean_frame = pd.read_csv('clean_frame.csv', encoding='utf-8',sep='|')

In [12]:
res_frame = pd.merge(clean_frame, addresses_frame,on="Адрес",how='left')

In [13]:
res_frame['Округ'] = res_frame['Полный ответ'].apply(search_district)

In [14]:
res_frame['Check district'] = res_frame[['Округ']].apply(lambda x: 1 if ',' in str(x.values) else 0, axis=1)

In [15]:
ressss = res_frame[res_frame['Check district']==0]

Тематическое моделирование

In [ ]:
df = pd.read_csv('clean_frame.csv',encoding='utf-8',sep='|')

In [ ]:
data = df['Obr_clean'].values.tolist()

In [ ]:
data_words = [nltk.word_tokenize(str(line)) for line in tqdm(data)]

In [ ]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
num_topics = 10 #количество тем
lda_model = gensim.models.LdaMulticore(corpus = corpus, id2word=id2word, num_topics = num_topics)

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

Работа с картой

In [3]:
import numpy as np
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
sns.set(style="whitegrid", palette="pastel", color_codes=True)
sns.mpl.rc("figure", figsize=(10,6))

In [5]:
shp_path = "for maps//mo.shp"
sf = shp.Reader(shp_path)

In [6]:
def read_shapefile(sf):
    fields = [x[0] for x in sf.fields][1:]#fetching the records from the shape file
    records = [list(i) for i in sf.records()]
    shps = [s.points for s in sf.shapes()]#converting shapefile data into pandas dataframe
    df = pd.DataFrame(columns=fields, data=records)#assigning the coordinates
    df = df.assign(coords=shps)
    return df

In [7]:
df = read_shapefile(sf)

In [8]:
df['NAME'] = df['NAME'].apply( lambda x: x.replace('ё', 'е') )

In [9]:
df['NAME'] = df['NAME'].apply(lambda x: x.capitalize())

In [16]:
ressss['NAME'] = ressss['Округ'].apply(lambda x: x.capitalize())

In [42]:
districts_values = ressss['NAME'].value_counts().to_frame()
df_districts_counts = pd.DataFrame(districts_values)
df_districts_counts = df_districts_counts.reset_index()
df_districts_counts.columns = ['NAME','COUNT']

In [34]:
rez = pd.merge(df,df_districts_counts, on='NAME', how='left')

In [39]:
rez[['COUNT']] = rez[['COUNT']].fillna(value=0)

In [52]:
rez.head()

,NAME,OKATO,OKTMO,NAME_AO,OKATO_AO,ABBREV_AO,TYPE_MO,coords,COUNT
0,Киевский,45298555,45945000,Троицкий,45298000,Троицкий,Поселение,"[(36.8031012, 55.4408329), (36.8031903, 55.441...",0.0
1,Филевский парк,45268595,45328000,Западный,45268000,ЗАО,Муниципальный округ,"[(37.4276499, 55.7482092), (37.4284863, 55.748...",593.0
2,Новофедоровское,45298567,45954000,Троицкий,45298000,Троицкий,Поселение,"[(36.8035692, 55.4516224), (36.8045117, 55.465...",0.0
3,Роговское,45298575,45956000,Троицкий,45298000,Троицкий,Поселение,"[(36.9372397, 55.2413907), (36.9372604, 55.241...",0.0
4,"""мосрентген""",45297568,45953000,Новомосковский,45297000,Новомосковский,Поселение,"[(37.4395575, 55.6273129), (37.4401803, 55.630...",0.0


In [ ]:
def plot_map(sf, x_lim = None, y_lim = None, figsize = (30,15)):
    plt.figure(figsize = figsize)
    id=0
    for shape in sf.shapeRecords():
        x = [i[0] for i in shape.shape.points[:]]
        y = [i[1] for i in shape.shape.points[:]]
        plt.plot(x, y, 'k')
        if (x_lim == None) & (y_lim == None):
            x0 = np.mean(x)
            y0 = np.mean(y)
            plt.text(x0, y0, id, fontsize=10)
        id = id+1
    if (x_lim != None) & (y_lim != None):     
        plt.xlim(x_lim)
        plt.ylim(y_lim)

In [ ]:
plot_map(sf)

In [70]:
def calc_color(data, color=None):
        if color   == 1: 
            color_sq =  ['#dadaebFF','#bcbddcF0','#9e9ac8F0','#807dbaF0','#6a51a3F0','#54278fF0']; 
            colors = 'Purples';
        elif color == 2: 
            color_sq = ['#c7e9b4','#7fcdbb','#41b6c4','#1d91c0','#225ea8','#253494']; 
            colors = 'YlGnBu';
        elif color == 3: 
            color_sq = ['#f7f7f7','#d9d9d9','#bdbdbd','#969696','#636363','#252525']; 
            colors = 'Greys';
        elif color == 9: 
            color_sq = ['#ff0000','#ff0000','#ff0000','#ff0000','#ff0000','#ff0000'];            
        else:           
            color_sq = ['#ffffd4','#fee391','#fec44f','#fe9929','#d95f0e','#993404']; 
            colors = 'YlOrBr';
        new_data, bins = pd.qcut(data, 6, retbins=True, labels=list(range(6)))
        color_ton = []
        for val in new_data:
            color_ton.append(color_sq[val]) 
        if color != 9:
            colors = sns.color_palette(colors, n_colors=6)
            sns.palplot(colors, 0.6);   
        return color_ton, bins;

def plot_cities_data(sf,
                    title,
                    cities,
                    data=None,
                    color=None,
                    print_id=False):
    color_ton, bins = calc_color(data, color)
    #df = read_shapefile(sf)
    df = sf
    city_id = []
    for i in cities:
        #city_id.append(df[df.NAME == i.upper()].index.get_values()[0])
        print(df[df.NAME == i.upper()].index)
    plot_map_fill_multiples_ids_tone(sf,
                                    title,
                                    city_id,
                                    print_id,
                                    color_ton,
                                    bins,
                                    x_lim = None,
                                    y_lim = None,
                                    figsize = (11,9))

def plot_map_fill_multiples_ids_tone(sf,
                                    title,
                                    city,
                                    print_id,
                                    color_ton,
                                    bins,
                                    x_lim = None,
                                    y_lim = None,
                                    figsize = (11,9)):
    plt.figure(figsize = figsize)
    fig, ax = plt.subplots(figsize = figsize)
    fig.suptitle(title, fontsize=16)
    for shape in sf.shapeRecords():
        x = [i[0] for i in shape.shape.points[:]]
        y = [i[1] for i in shape.shape.points[:]]
        ax.plot(x, y, 'k')
            
    for id in city:
        shape_ex = sf.shape(id)
        x_lon = np.zeros((len(shape_ex.points),1))
        y_lat = np.zeros((len(shape_ex.points),1))
        for ip in range(len(shape_ex.points)):
            x_lon[ip] = shape_ex.points[ip][0]
            y_lat[ip] = shape_ex.points[ip][1]
        ax.fill(x_lon,y_lat, color_ton[city.index(id)])
        if print_id != False:
            x0 = np.mean(x_lon)
            y0 = np.mean(y_lat)
            plt.text(x0, y0, id, fontsize=10)
    if (x_lim != None) & (y_lim != None):     
        plt.xlim(x_lim)
        plt.ylim(y_lim)

In [71]:
res_res = rez[rez['COUNT']!= 0.0]

In [72]:
print_id = True # The shape id will be printed
color_pallete = 1
rez['NAME'] = rez['NAME'].astype(str)
names= res_res['NAME'].tolist()
data = res_res['COUNT'].tolist()
plot_cities_data(rez, 'Heat map of given cities', names, data, color_pallete, print_id)

Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index

AttributeError: 'DataFrame' object has no attribute 'shapeRecords'